In [1]:
import numpy as np
import tensorflow as tf
import os

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/notebooks/data/MNIST_data/", one_hot
=True)

In [ ]:
print(mnist.validation.labels[:1])
print('Training set', mnist.train.images.shape, mnist.train.labels.shape
)
print('Validation set', mnist.validation.images.shape, mnist.validation.
labels.shape)
print('Test set', mnist.test.images.shape, mnist.test.labels.shape)

In [ ]:
 def hidden(data, in_dims, out_dims, name):
 with tf.name_scope(name):
 w = tf.Variable(tf.truncated_normal([in_dims, out_dims], stddev=
0.01), name="W")
 #w = tf.Variable(tf.zeros([in_dims, out_dims]), name="W")
 b = tf.Variable(tf.zeros([out_dims]), name="B")
 out = tf.matmul(data, w) + b
 act = tf.nn.relu(out)
 return act


In [ ]:
def out_layer(data, in_dims, out_dims, name):
 with tf.name_scope(name):
 w = tf.Variable(tf.truncated_normal([in_dims, out_dims], stddev=
0.01), name="W")
 #w = tf.Variable(tf.zeros([in_dims, out_dims]), name="W")
 b = tf.Variable(tf.zeros(out_dims))
 out = tf.matmul(data, w) + b
 return out


In [ ]:
batch_size = 50
num_of_epochs = 4
image_size = 28
num_labels = 10
log_dir = '/notebooks/data/demo/'
lr = 1e-2

In [ ]:
run_name = 'run-demo-2-lr-' + str(lr) + '-b' + str(batch_size) + 'h_' +
'L-100-150-50-25'

In [ ]:
 tf.reset_default_graph()


In [ ]:
x = tf.placeholder(tf.float32, shape=[None, image_size * image_size], na
me="X")
y_label = tf.placeholder(tf.float32, shape=[None, num_labels], name="Y_l
abel")
image_shaped_input = tf.reshape(x, [-1, image_size, image_size, 1])
tf.summary.image('input', image_shaped_input, 10)


In [ ]:
hidden1 = hidden(x, image_size * image_size, 100, "hidden1")
hidden2 = hidden(hidden1, 100, 150, "hidden2")
hidden3 = hidden(hidden2, 150, 50, "hidden3")
hidden4 = hidden(hidden3, 50, 25, "hidden4")
pred = out_layer(hidden4, 25, num_labels, "pred")


In [ ]:
 with tf.name_scope("xent"):
 cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits
=pred, labels=y_label))
with tf.name_scope("accuracy"):
 correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y_label, 1))
 accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
tf.summary.scalar('acc', accuracy)
tf.summary.scalar('loss', cost)


In [ ]:
with tf.name_scope("train"):
 train_step = tf.train.AdamOptimizer(lr).minimize(cost)


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [ ]:
 summ = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + 'train/' + run_name)
train_writer.add_graph(sess.graph)


In [ ]:
total_train_indexes = len(mnist.train.images)
for epoch in range(num_of_epochs):
 i = 0
 epoch_loss = 0
 while i < total_train_indexes:
 start = i
 end = i + batch_size
 if end > total_train_indexes:
 end = total_train_indexes
 batch_dataset = mnist.train.images[start:end]
 batch_labels = mnist.train.labels[start:end]
 input_dict = {x: batch_dataset, y_label: batch_labels}
 _, epoch_accuracy, loss, s = sess.run([train_step, accuracy, cos
t, summ], feed_dict=input_dict)
 epoch_loss += loss
 train_writer.add_summary(s, i + total_train_indexes * epoch)
 i += batch_size
 valid_accuracy, valid_loss = sess.run([accuracy, cost],
 feed_dict={x: mnist.v
alidation.images,
 y_label: m
nist.validation.labels})

 print('Epoch', epoch + 1, 'completed out of',num_of_epochs,'train_lo
ss:',epoch_loss, 'train_ccuracy:', epoch_accuracy)
 print('Epoch', epoch + 1, 'completed out of',num_of_epochs,'valid_lo
ss:',valid_loss, 'valid_accuracy:', valid_accuracy)

test_accuracy, test_loss = sess.run([accuracy, cost], feed_dict={x: mnis
t.test.images, y_label: mnist.test.labels})
print("Test accuracy:", test_accuracy, "Test loss:", test_loss)


In [ ]:
from IPython.display import Image
Image("/notebooks/data/images/tensorboard.png")